In [1]:
%load_ext autoreload
%autoreload 2

from utils import BPETokenizer

import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb

In [9]:
train_files = open("data/PY150K/python100k_train.txt", "r").read().split("\n")
train_texts = [open("data/PY150K/" + path, encoding='iso-8859-1').read() for path in train_files]

tokenizer = BPETokenizer(initial_tokens="\n".join(train_texts)) # ensure that all unary tokens in our training data exist in our vocabulary
tokenizer.fit("\n".join(train_texts[:1000]), 100) # fit the BPE on only the first 1000 files for computational reasons
tokenizer.save("py150k_large")
#tokenizer = BPETokenizer.load("py150k_large")

IsADirectoryError: [Errno 21] Is a directory: 'data/PY150K/'

In [8]:
tokenizer.print_tokens(open("data/PY150K/" + train_files[10]).read())

# -*- coding: utf-8 -*-
from __future__ import unicode_literals

from django.db import models, migrations


class Migration(migrations.Migration):

    dependencies = [
        ('testreport', '0026_testresult_launch_item_id'),
    ]

    operations = [
        migrations.AddField(
            model_name='testplan',
            name='filter',
            field=models.TextField(default=b'', max_length=128, verbose_name='Started by filter', blank=True),
            preserve_default=True,
        ),
        migrations.AddField(
            model_name='testplan',
            name='main',
            field=models.BooleanField(default=False, verbose_name='Show in short statistic'),
            preserve_default=True,
        ),
    ]


In [ ]:
from utils.dataset import PY150kDataset

ds = PY150kDataset("train", "py150k_large")
ds[1337]

RuntimeError: 

One problem is that we need all sequences in a batch to be the same length, but there is a large difference in lengths

In [6]:
max(len(x) for x in ds), min(len(x) for x in ds)

RuntimeError: 

Having 14 tokens is way to little, let's drop training examples where we have fever than 100 tokens

In [7]:
train_tokens = [seq for seq in train_tokens if len(seq) > 100]
len(train_tokens)

980

In [8]:
def collate_fn(batch:list[list[int]]):
    min_length = min(len(x) for x in batch)
    return torch.tensor([x[:min_length] for x in batch])

class ListDataset(Dataset): # simply defines how you access your data, trivial here but perhaps not later
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

train_ds = ListDataset(train_tokens)
train_dl = DataLoader(train_ds, batch_size=32, collate_fn=collate_fn, shuffle=True, prefetch_factor=4, num_workers=8)

For example purposes this will be a many-to-one encoder-decoder architecture. Our transformer atleast will probably be decoder only.

In [9]:
class PyLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x:torch.Tensor)->torch.Tensor:
        x = self.embed(x) # vocab to hidden
        x, (h_n, c_n) = self.rnn(x) # rnn also returns it's hidden/cell state
        x = self.linear(F.relu(x[:,-1])) # hidden to vocab
        return F.softmax(x)
    
model = PyLSTM(len(tokenizer), 128)
model(next(iter(train_dl))).shape

/tmp/ipykernel_157805/2546379325.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


torch.Size([32, 240])

https://wandb.ai/bjarnih/PyGPT

In [13]:
from tqdm import tqdm
import wandb

EPOCHS = 10
LR = 3e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

wandb.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": LR,
        "epochs": EPOCHS,
        "architecture": "many-to-one LSTM",
        "dataset": "small subset of PY150k",
    },
)

model = PyLSTM(len(tokenizer), 128).to(DEVICE)
optim = torch.optim.Adam(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss().to(DEVICE)

for i in range(EPOCHS):
    for batch in tqdm(train_dl):
        x = batch[..., :-1].to(DEVICE).float()
        y = batch[..., -1].to(DEVICE) # keeps the unary dimension

        y_hat = model(x)

        loss = loss_fn(y, y_hat)

        optim.zero_grad()
        loss.backward()
        optim.step()

        wandb.log({"train_loss": loss.detach().numpy()})

  0%|          | 0/31 [00:00<?, ?it/s]ERROR:tornado.general:SEND Error: Host unreachable
